# LIBERO Policy Evaluation

This notebook evaluates a policy on LIBERO benchmark tasks.


## 1. Import Libraries


In [ ]:
import collections
import dataclasses
import logging
import math, sys, os
import pathlib
from datetime import datetime

sys.path.append("/hdd/zijianwang/openpi/third_party/LIBERO-PRO")

import imageio
from libero.libero import benchmark
from libero.libero import get_libero_path
from libero.libero.envs import OffScreenRenderEnv
import numpy as np
from openpi_client import image_tools
from openpi_client import websocket_client_policy as _websocket_client_policy
import tqdm
import matplotlib.pyplot as plt

# Setup logging
logging.basicConfig(level=logging.INFO)


## 2. Define Constants and Helper Functions


In [ ]:
LIBERO_DUMMY_ACTION = [0.0] * 6 + [-1.0]
LIBERO_ENV_RESOLUTION = 256  # resolution used to render training data


def _get_libero_env(task, resolution, seed):
    """Initializes and returns the LIBERO environment, along with the task description."""
    task_description = task.language
    task_bddl_file = pathlib.Path(get_libero_path("bddl_files")) / task.problem_folder / task.bddl_file
    env_args = {"bddl_file_name": task_bddl_file, "camera_heights": resolution, "camera_widths": resolution}
    env = OffScreenRenderEnv(**env_args)
    env.seed(seed)  # IMPORTANT: seed seems to affect object positions even when using fixed initial state
    return env, task_description


def _quat2axisangle(quat):
    """
    Copied from robosuite: https://github.com/ARISE-Initiative/robosuite/blob/eafb81f54ffc104f905ee48a16bb15f059176ad3/robosuite/utils/transform_utils.py#L490C1-L512C55
    """
    # clip quaternion
    if quat[3] > 1.0:
        quat[3] = 1.0
    elif quat[3] < -1.0:
        quat[3] = -1.0

    den = np.sqrt(1.0 - quat[3] * quat[3])
    if math.isclose(den, 0.0):
        # This is (close to) a zero degree rotation, immediately return
        return np.zeros(3)

    return (quat[:3] * 2.0 * math.acos(quat[3])) / den


def _plot_velocity_trajectory(velocity_trajectory, output_path):
    """
    Plot velocity trajectory with 7 dimensions as different colored lines.
    
    Args:
        velocity_trajectory: List of 7-dimensional velocity vectors
        output_path: Path to save the plot
    """
    if len(velocity_trajectory) == 0:
        logging.warning("No velocity data to plot")
        return
    
    velocity_array = np.array(velocity_trajectory)  # Shape: (T, 7)
    time_steps = np.arange(len(velocity_trajectory))
    
    # Create figure with good size
    plt.figure(figsize=(12, 6))
    
    # Define colors for 7 dimensions
    colors = ['red', 'blue', 'green', 'orange', 'purple', 'brown', 'pink']
    dimension_labels = [f'Joint {i+1}' for i in range(7)]
    
    # Plot each dimension with different color
    for dim in range(7):
        plt.plot(time_steps, velocity_array[:, dim], 
                color=colors[dim], label=dimension_labels[dim], linewidth=1.5, alpha=0.8)
    
    plt.xlabel('Time Step', fontsize=12)
    plt.ylabel('Velocity (rad/s)', fontsize=12)
    plt.title('Joint Velocity Trajectory', fontsize=14, fontweight='bold')
    plt.legend(loc='best', fontsize=10)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    
    # Save the plot
    plt.savefig(output_path, dpi=100)
    plt.close()
    
    logging.info(f"Velocity plot saved to: {output_path}")


## 3. Set Hyperparameters


In [7]:
@dataclasses.dataclass
class Args:
    #################################################################################################################
    # Model server parameters
    #################################################################################################################
    host: str = "0.0.0.0"
    port: int = 8001
    resize_size: int = 224
    replan_steps: int = 5

    #################################################################################################################
    # LIBERO environment-specific parameters
    #################################################################################################################
    task_suite_name: str = (
        "libero_10"  # Task suite. Options: libero_spatial, libero_object, libero_goal, libero_10, libero_90
    )
    num_steps_wait: int = 10  # Number of steps to wait for objects to stabilize in sim
    num_trials_per_task: int = 50  # Number of rollouts per task

    #################################################################################################################
    # Utils
    #################################################################################################################
    video_out_path: str = "data/libero/videos"  # Path to save videos

    seed: int = 7  # Random Seed (for reproducibility)


# Create args instance - you can modify these values as needed
args = Args(
    host="0.0.0.0",
    port=8001,
    resize_size=224,
    replan_steps=5,
    task_suite_name="libero_10",
    num_steps_wait=10,
    num_trials_per_task=50,
    video_out_path="data/libero/videos",
    seed=7
)

print("Hyperparameters:")
print(f"  Host: {args.host}")
print(f"  Port: {args.port}")
print(f"  Resize size: {args.resize_size}")
print(f"  Replan steps: {args.replan_steps}")
print(f"  Task suite: {args.task_suite_name}")
print(f"  Num steps wait: {args.num_steps_wait}")
print(f"  Num trials per task: {args.num_trials_per_task}")
print(f"  Video output path: {args.video_out_path}")
print(f"  Seed: {args.seed}")


Hyperparameters:
  Host: 0.0.0.0
  Port: 8001
  Resize size: 224
  Replan steps: 5
  Task suite: libero_10
  Num steps wait: 10
  Num trials per task: 50
  Video output path: data/libero/videos
  Seed: 7


## 4. Run Policy Evaluation

**Note:** Make sure your model server is running before executing this cell!


In [ ]:
# Set random seed
np.random.seed(args.seed)

# Get current timestamp for this run
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# Create timestamped video output directory
video_out_dir = pathlib.Path(args.video_out_path) / timestamp / args.task_suite_name
video_out_dir.mkdir(parents=True, exist_ok=True)
logging.info(f"Videos will be saved to: {video_out_dir}")

# Initialize LIBERO task suite
benchmark_dict = benchmark.get_benchmark_dict()
task_suite = benchmark_dict[args.task_suite_name]()
num_tasks_in_suite = task_suite.n_tasks
logging.info(f"Task suite: {args.task_suite_name}")

if "libero_spatial" in args.task_suite_name:
    max_steps = 220  # longest training demo has 193 steps
elif "libero_object" in args.task_suite_name:
    max_steps = 280  # longest training demo has 254 steps
elif "libero_goal" in args.task_suite_name:
    max_steps = 300  # longest training demo has 270 steps
elif "libero_10" in args.task_suite_name:
    max_steps = 520  # longest training demo has 505 steps
elif "libero_90" in args.task_suite_name:
    max_steps = 400  # longest training demo has 373 steps
else:
    raise ValueError(f"Unknown task suite: {args.task_suite_name}")

client = _websocket_client_policy.WebsocketClientPolicy(args.host, args.port)

# Start evaluation
total_episodes, total_successes = 0, 0
for task_id in tqdm.tqdm(range(num_tasks_in_suite)):
    # Get task
    task = task_suite.get_task(task_id)

    # Get default LIBERO initial states
    initial_states = task_suite.get_task_init_states(task_id)

    # Initialize LIBERO environment and task description
    env, task_description = _get_libero_env(task, LIBERO_ENV_RESOLUTION, args.seed)

    # Start episodes
    task_episodes, task_successes = 0, 0
    for episode_idx in tqdm.tqdm(range(args.num_trials_per_task)):
        if episode_idx != 0:
            sys.exit()

        # Reset environment
        env.reset()
        action_plan = collections.deque()

        # Set initial states
        obs = env.set_init_state(initial_states[episode_idx])

        # Setup
        t = 0
        replay_images = []
        velocity_trajectory = []  # Store velocity data for visualization

        logging.info(f"Starting episode {task_episodes+1}...")
        while t < max_steps + args.num_steps_wait:
            print(f"t: {t}")
            try:
                # IMPORTANT: Do nothing for the first few timesteps because the simulator drops objects
                # and we need to wait for them to fall
                if t < args.num_steps_wait:
                    obs, reward, done, info = env.step(LIBERO_DUMMY_ACTION)
                    t += 1
                    continue

                # Get preprocessed image
                # IMPORTANT: rotate 180 degrees to match train preprocessing
                img = np.ascontiguousarray(obs["agentview_image"][::-1, ::-1])
                wrist_img = np.ascontiguousarray(obs["robot0_eye_in_hand_image"][::-1, ::-1])
                img = image_tools.convert_to_uint8(
                    image_tools.resize_with_pad(img, args.resize_size, args.resize_size)
                )
                wrist_img = image_tools.convert_to_uint8(
                    image_tools.resize_with_pad(wrist_img, args.resize_size, args.resize_size)
                )

                # Save preprocessed image for replay video
                replay_images.append(img)

                if not action_plan:
                    # Finished executing previous action chunk -- compute new chunk
                    # Prepare observations dict
                    element = {
                        "observation/image": img,
                        "observation/wrist_image": wrist_img,
                        "observation/state": np.concatenate(
                            (
                                obs["robot0_eef_pos"],
                                _quat2axisangle(obs["robot0_eef_quat"]),
                                obs["robot0_gripper_qpos"],
                            )
                        ),
                        "prompt": str(task_description),
                    }

                    # Query model to get action
                    action_chunk = client.infer(element)["actions"]
                    assert (
                        len(action_chunk) >= args.replan_steps
                    ), f"We want to replan every {args.replan_steps} steps, but policy only predicts {len(action_chunk)} steps."
                    action_plan.extend(action_chunk[: args.replan_steps])

                action = action_plan.popleft()
                # print(f"action: {action}")

                # Execute action in environment
                obs, reward, done, info = env.step(action.tolist())
                
                # Collect velocity data
                velocity = obs["robot0_joint_vel"]
                velocity_trajectory.append(velocity)
                print(velocity)
                action_length = len(action.tolist())
                if done:
                    task_successes += 1
                    total_successes += 1
                    break
                t += 1

            except Exception as e:
                logging.error(f"Caught exception: {e}")
                break

        task_episodes += 1
        total_episodes += 1

        # Save a replay video of the episode with unique filename
        # Include task_id, episode_idx, and success/failure status
        suffix = "success" if done else "failure"
        task_segment = task_description.replace(" ", "_")
        video_filename = f"task{task_id:02d}_ep{episode_idx:03d}_{task_segment}_{suffix}.mp4"
        video_path = video_out_dir / video_filename
        
        imageio.mimwrite(
            video_path,
            [np.asarray(x) for x in replay_images],
            fps=24,
        )
        logging.info(f"Video saved to: {video_path}")
        
        # Save velocity trajectory plot
        velocity_plot_filename = f"task{task_id:02d}_ep{episode_idx:03d}_{task_segment}_{suffix}_velocity.png"
        velocity_plot_path = video_out_dir / velocity_plot_filename
        _plot_velocity_trajectory(velocity_trajectory, str(velocity_plot_path))

        # Log current results
        logging.info(f"Success: {done}")
        logging.info(f"# episodes completed so far: {total_episodes}")
        logging.info(f"# successes: {total_successes} ({total_successes / total_episodes * 100:.1f}%)")

    # Log final results
    logging.info(f"Current task success rate: {float(task_successes) / float(task_episodes)}")
    logging.info(f"Current total success rate: {float(total_successes) / float(total_episodes)}")

logging.info(f"Total success rate: {float(total_successes) / float(total_episodes)}")
logging.info(f"Total episodes: {total_episodes}")
logging.info(f"All videos saved to: {video_out_dir}")


INFO:root:Videos will be saved to: data/libero/videos/20251027_142223/libero_10
INFO:root:Task suite: libero_10
INFO:root:Waiting for server at ws://0.0.0.0:8001...


[info] Applying task order index 0 (permutation: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]) for benchmark 'libero_10' (10 tasks).


  0%|          | 0/10 [00:00<?, ?it/s]

[Warning]: datasets path /hdd/zijianwang/openpi/third_party/LIBERO-PRO/libero/libero/../datasets does not exist!
[Warning]: datasets path /hdd/zijianwang/openpi/third_party/LIBERO-PRO/libero/libero/../datasets does not exist!


INFO:root:Starting episode 1...


t: 0
t: 1
t: 2
t: 3
t: 4
t: 5
t: 6
t: 7
t: 8
t: 9
t: 10
[-0.00048218  0.06237037 -0.05884652  0.07716406 -0.0450679  -0.07567992
 -0.01447865]
t: 11
[-0.00414898  0.07106534 -0.07359909  0.08470453 -0.04246332 -0.07724525
 -0.03840919]
t: 12
[-0.00745541  0.07507937 -0.08519884  0.09037754 -0.04093722 -0.08136168
 -0.05688821]
t: 13
[-0.01166227  0.07737198 -0.09865829  0.09461451 -0.04118548 -0.08582214
 -0.07548856]
t: 14
[-0.01686049  0.07937535 -0.11463865  0.09806087 -0.04313806 -0.08724036
 -0.09443902]
t: 15
[-0.02264463  0.08359123 -0.13219166  0.10707236 -0.04645288 -0.09345769
 -0.11654989]
t: 16
[-0.02875755  0.08935329 -0.15049211  0.1152056  -0.05027748 -0.10291134
 -0.13760673]
t: 17
[-0.03581333  0.09673991 -0.17180706  0.12477215 -0.05513724 -0.10789028
 -0.15862837]
t: 18
[-0.04352866  0.105803   -0.1955249   0.13358514 -0.06019723 -0.10951159
 -0.18316716]
t: 19
[-0.05137337  0.11599237 -0.22017965  0.14283658 -0.06249054 -0.10915685
 -0.21200541]
t: 20
[-0.06222473  

INFO:root:Video saved to: data/libero/videos/20251027_142223/libero_10/task00_ep000_put_both_the_alphabet_soup_and_the_tomato_sauce_in_the_basket_success.mp4
INFO:root:Success: True
INFO:root:# episodes completed so far: 1
INFO:root:# successes: 1 (100.0%)
  0%|          | 0/10 [00:11<?, ?it/s]


SystemExit: 

/hdd/zijianwang/openpi/examples/libero/.venv/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
